In [1]:
import cv2
import numpy as np

# Codigo de exemplo para extrair uma pessoa do fundo usando contornos e morfologia

# Open webcam
vidBuffer = cv2.VideoCapture(0)
framerate = 60
delayFrame = int(1.0 / framerate * 1000)

while True:
    flag, img = vidBuffer.read()
    if not flag:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use Otsu threshold for automatic separation
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations to clean noise & fill holes
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Build mask
    mask = np.zeros(img.shape[:2], np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:  # ignore tiny blobs
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)

    mask = cv2.bitwise_not(mask)  # Invert mask


    # Invert mask
    mask_inv = cv2.bitwise_not(mask)

    # Extract person
    fg = cv2.bitwise_and(img, img, mask=mask)
    
    cv2.imshow("Person", fg)

    key = cv2.waitKey(delayFrame)
    if key == 27:  # ESC to exit
        break

vidBuffer.release()
cv2.destroyAllWindows()


In [2]:
def extractPerson(frame):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Use Otsu threshold for automatic separation
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological operations to clean noise & fill holes
    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Build mask
    mask = np.zeros(img.shape[:2], np.uint8)
    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:  # ignore tiny blobs
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)

    mask = cv2.bitwise_not(mask)  # Invert mask

    # Extract person
    fg = cv2.bitwise_and(img, img, mask=mask)
    return fg

In [3]:
def AugmentedReality(video0, ini, fin): 
    # Abre o vídeo original
    original = cv2.VideoCapture(video0)
    
    # Prepara o vídeo cortado
    cut_name = "cut_video.avi"
    height = int(original.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(original.get(cv2.CAP_PROP_FRAME_WIDTH))
    
    # Define o codec e cria o VideoWriter para o vídeo cortado
    videoWriter = cv2.VideoWriter(cut_name, -1, original.get(cv2.CAP_PROP_FPS), (width, height), 0)
    
    if (ini >= 0 and fin >= 0):
    # Cortar o video entre os frames ini e fin
        for i in range(original.get(cv2.CAP_PROP_FRAME_COUNT)):
            ret, frame = original.read()
            if not ret:
                break
            if i >= ini and i <= fin:
                videoWriter.write(frame)
    """
    Common Issues:
    Codec not supported: If the specified codec is not supported by your system, you may see an error message. 
    In this case, you can try using a different codec or adjusting the fourcc parameter.
    Insufficient memory: If the video is too large, you may run out of memory. To mitigate this issue, you can try 
    reducing the frame rate or using a more efficient codec.
    Incorrect frame rate: If the frame rate is incorrect, you may see a jerky or choppy video. To fix this issue, 
    you can adjust the fps parameter or use a different codec.
    """

    
    # NÃO É PARA USAR HOG, CASCADE OU OUTRAS BLIBLIOTECAS DE DETEÇÃO DE PESSOAS/ROSTOS/OBJETOS
    
    # Para a deteção e classificação de partes do corpo temos que:
    # extrair o corpo humano do fundo (usando o código acima como base)
    # encontrar o centroide do corpo
    # 2 OPÇÕES:
        # ver a cor de pele e ver a parte em que essa cor está mais a esquerda/direita do centroide
        # ou a partir dos contornos ver o ponto mais à esquerda/direita do centroide (melhor)
        
    # Com essa informação podemos fazer a agua a voar para o braço OU fazemos uma mini versão de ti aparecer na tua mão
    
    # Para cada frame do vídeo cortado entre ini e fin:
        # Detectar a pessoa no frame
            # Extrair a região do braço
            # Pôr a aguia/pessoa no braco/mao detetado(a)
    
    # Salvar o vídeo processado como video1.mp4
    
    # Retorna uma String com o nome do arquivo de vídeo processado
    return "video1.avi"

video0 = "./video0.avi"
ini = 0 # Frame inicial
fin = 10 # Frame final
video1 = AugmentedReality(video0 ,ini, fin)


TypeError: 'float' object cannot be interpreted as an integer

In [8]:
def findFurthestPointFromCentroid(contours, centroid):
    cnt = max(contours, key=cv2.contourArea)  # Maior contorno
    
    pts = cnt.reshape(-1, 2)  # Converter contorno para array de pontos 2D
    # Encontrar ponto mais distante do centroide (mão)
    dx = pts[:, 0] - centroid[0] # Lista das distancias de x ao centroide
    dy = pts[:, 1] - centroid[1] # Lista das distancias de y ao centroide
    dists = np.hypot(dx, dy) # Lista das distancias euclidianas ao centroide

    max_idx = np.argmax(dists) # Indice do ponto mais distante
    hand_x, hand_y = pts[max_idx] # Coordenadas do ponto mais distante (mão)
    hand_dist = dists[max_idx] # Distancia até ao ponto mais distante (mão)
    return (hand_x, hand_y), hand_dist

In [9]:
import cv2
import numpy as np

# Codigo de exemplo para extrair uma pessoa do fundo usando contornos e morfologia

# Open webcam
vidBuffer = cv2.VideoCapture("./video0.avi")
framerate = 60
delayFrame = int(1.0 / framerate * 1000)

cv2.namedWindow("Mask", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Mask", 500, 300)

cv2.namedWindow("Contours", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Contours", 500, 300)

cv2.namedWindow("Person", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Person", 500, 300)

while True:
    flag, img = vidBuffer.read()
    if not flag:
        break

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    kernel = np.ones((5, 5), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2)

    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros(img.shape[:2], np.uint8)
    contour_vis = img.copy()   # visualization window

    for cnt in contours:
        if cv2.contourArea(cnt) > 5000:
            cv2.drawContours(mask, [cnt], -1, 255, thickness=-1)
            cv2.drawContours(contour_vis, [cnt], -1, (0, 255, 0), 2)

    cv2.imshow("Mask", mask)
    cv2.imshow("Contours", contour_vis)

    inv_mask = cv2.bitwise_not(mask)
    
    M = cv2.moments(inv_mask)
    
    img_width = img.shape[1]
 
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    
    cv2.circle(img, (cX, cY), 5, (255, 255, 255), -1)
    cv2.putText(img, "centroid", (cX - 25, cY - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    hand_coords, hand_dist = findFurthestPointFromCentroid(contours, (cX, cY))
    
    cv2.circle(img, hand_coords, 5, (0, 0, 255), -1)
    cv2.putText(img, "hand", (hand_coords[0] - 25, hand_coords[1] - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    fg = cv2.bitwise_and(img, img, mask=inv_mask)

    cv2.imshow("Person", fg)
    key = cv2.waitKey(delayFrame)
    if key == 27:  # ESC to exit
        break

vidBuffer.release()
cv2.destroyAllWindows()
